In [ ]:
# Install required libraries
%pip install transformers gradio torch

In [ ]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the pre-trained conversational model and tokenizer
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
def predict(message, history):
    """
    Generate a response from the model.
    Note: For a real legal/insurance bot, we would inject context or use a RAG pipeline here.
    """
    # Encode the user input
    new_user_input_ids = tokenizer.encode(message + tokenizer.eos_token, return_tensors='pt')

    # Generate a response
    # We limit max_length to keep it quick for the demo
    bot_input_ids = new_user_input_ids
    chat_history_ids = model.generate(
        bot_input_ids, 
        max_length=1000, 
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True, 
        top_k=100, 
        top_p=0.7,
        temperature=0.8
    )

    # Decode the response (skip the input tokens)
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    if not response:
        return "I'm not sure what to say to that."
        
    return response

In [ ]:
# Create and launch the Gradio Chat Interface
demo = gr.ChatInterface(
    predict,
    title="Dignit AI Model Demo",
    description="A demo of the conversational model. Ask about general topics (Note: This base model is not yet fine-tuned for legal advice).",
    examples=["Hello, how are you?", "What is the process for asylum?", "Tell me about insurance."]
)

demo.launch()